#Environment setup — install ADK & verify CLI


In [1]:
import sys
print("Python:", sys.version)

# Install/upgrade ADK
!pip -q install --upgrade google-adk

# Verify CLI is available
!adk --help | tail -n +20


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


#Scaffold A1 prototype agent (kitchen renovation, no tools)


In [3]:
%%bash
set -e
rm -rf a1_prototype_agent
mkdir -p a1_prototype_agent

cat > a1_prototype_agent/__init__.py << 'PY'
from .agent import root_agent
PY

cat > a1_prototype_agent/agent.py << 'PY'
from google.adk.agents import Agent

# A1 domain agent: generates a concise renovation proposal as Markdown
root_agent = Agent(
    model='gemini-2.0-flash',
    name='a1_prototype_agent',
    description='Generates a kitchen renovation proposal.',
    instruction=(
        "You are a renovation proposal assistant. Given inputs (home type, style, rooms, budget, timeline), "
        "produce a professional, concise proposal with these sections: "
        "1) Scope of Work, 2) Materials & Fixtures, 3) Phased Timeline, 4) Budget Breakdown (Markdown table), "
        "5) Risks & Mitigations, 6) Next Steps. Keep it ~400–600 words. Use clear Markdown headings."
    ),
    tools=[],   # A1: no tools yet
)
PY

echo "[OK] a1_prototype_agent ready"
ls -la a1_prototype_agent


[OK] a1_prototype_agent ready
total 16
drwxr-xr-x 2 root root 4096 Oct 20 02:39 .
drwxr-xr-x 1 root root 4096 Oct 20 02:39 ..
-rw-r--r-- 1 root root  710 Oct 20 02:39 agent.py
-rw-r--r-- 1 root root   30 Oct 20 02:39 __init__.py


#Run agent → save proposal (Markdown) + optional Drive copy + preview


In [4]:
import subprocess, re, os, shutil
from datetime import datetime

def run_agent(agent_pkg: str, prompt: str) -> str:
    p = subprocess.run(
        ["adk", "run", agent_pkg],
        input=(prompt + "\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=False,
    )
    out = p.stdout.decode("utf-8")
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*)", out, flags=re.S)
    return m.group(1).strip() if m else out

# --- sample inputs you can tweak ---
inputs = dict(
    home_type="condo",
    style="modern-minimal",
    rooms="kitchen only",
    budget="USD 25,000",
    timeline="6 weeks",
    must_haves="quartz countertop, induction cooktop, under-cabinet lighting",
)

prompt = f"""Create a kitchen renovation proposal using these inputs:

- Home type: {inputs['home_type']}
- Style: {inputs['style']}
- Scope: {inputs['rooms']}
- Budget: {inputs['budget']}
- Timeline: {inputs['timeline']}
- Must-haves: {inputs['must_haves']}

Return Markdown only.
"""

md = run_agent("a1_prototype_agent", prompt)

# Save locally
stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
fname = f"Kitchen_Renovation_Proposal_{stamp}.md"
with open(fname, "w", encoding="utf-8") as f:
    f.write(md)
print("Saved:", fname)

# If Google Drive is mounted, copy there too (free alternative to Cloud Storage)
drive_root = "/content/drive/MyDrive"
if os.path.isdir(drive_root):
    dest = os.path.join(drive_root, fname)
    shutil.copy(fname, dest)
    print("Copied to Drive:", dest)

# Show first 25 lines for validation
print("\n--- Preview ---")
print("\n".join(md.splitlines()[:25]))


Saved: Kitchen_Renovation_Proposal_20251020_023954.md

--- Preview ---
Okay, I can help with that. Please provide the inputs (home type, style, rooms, budget, timeline) so I can generate a kitchen renovation proposal for you. I need that information to tailor the proposal to your specific needs.

[user]: [user]: [a1_prototype_agent]: Okay, I will remember that we are working on a condo. What are the other inputs? (style, rooms, budget, timeline)

[user]: [a1_prototype_agent]: Okay, I have the home type (condo) and style (modern-minimal). I still need the following inputs to create the renovation proposal: rooms, budget, and timeline. Please provide those.

[user]: [a1_prototype_agent]: Okay, I have the home type (condo), style (modern-minimal), and scope (kitchen only). I still need the following inputs to create the renovation proposal: budget and timeline. Please provide those.

[user]: [a1_prototype_agent]: Okay, I have the home type (condo), style (modern-minimal), scope (kitchen o

#Sanity check — verify required sections exist


In [5]:
import re, pathlib

path = pathlib.Path(fname)  # from your save cell
text = path.read_text(encoding="utf-8")

need = [r"\bScope of Work\b",
        r"\bMaterials? & Fixtures\b",
        r"\bPhased Timeline\b",
        r"\bBudget Breakdown\b",
        r"\bRisks? & Mitigations\b",
        r"\bNext Steps\b"]

missing = [h for h in need if not re.search(h, text, flags=re.I)]
print("Missing sections:" if missing else "All sections present ✅", missing)


All sections present ✅ []
